# <center><a href='https://mybinder.org/v2/gh/fortierq/binder-mp2i/main?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fmp2i-fsm%252Fmp2i-2021%26urlpath%3Dlab%252Ftree%252Fmp2i-2021%252Fds%252Fds5%252Fcor%252Fcs21_cor.ipynb%26branch%3Dmain'>Arbres couvrants et pavages (Centrale Option info 2021)   <img src=https://mybinder.org/badge.svg></a></center>

## Quelques fonctions auxiliaires

In [1]:
(* 1. *)
let nombre_aretes g = 
    let res = ref 0 in
    for i = 0 to Array.length g - 1 do
        res := !res + List.length g.(i)
    done;
    !res/2;;

(* test sur le graphe de la figure 1 *)
let g1 = [| []; [3; 4]; [3]; [1; 2; 4; 4]; [1; 3; 3] |];;
nombre_aretes g1

val nombre_aretes : 'a list array -> int = <fun>


val g1 : int list array = [|[]; [3; 4]; [3]; [1; 2; 4; 4]; [1; 3; 3]|]


- : int = 5


In [2]:
(* 2. *)
let g32 = [|[|1; 3|]; [|0; 2; 4|]; [|1; 5|]; [|0; 4|]; [|1; 3; 5|]; [|2; 4|]|];;

val g32 : int array array =
  [|[|1; 3|]; [|0; 2; 4|]; [|1; 5|]; [|0; 4|]; [|1; 3; 5|]; [|2; 4|]|]


In [3]:
(* 3. *)
let adjacence = Array.map Array.of_list;;
let g32_list = [|[1; 3]; [0; 2; 4]; [1; 5]; [0; 4]; [1; 3; 5]; [2; 4]|];;
adjacence g32_list

val adjacence : '_weak1 list array -> '_weak1 array array = <fun>


val g32_list : int list array =
  [|[1; 3]; [0; 2; 4]; [1; 5]; [0; 4]; [1; 3; 5]; [2; 4]|]


- : int array array =
[|[|1; 3|]; [|0; 2; 4|]; [|1; 5|]; [|0; 4|]; [|1; 3; 5|]; [|2; 4|]|]


In [4]:
(* 4. *)
let rang (p, q) (s, t) =
    if t = s + 1 then (p*(q - 1) + s - s/4)
    else s mod 4 + (q - 1)*s/4;;
    
rang (4, 3) (6, 7)

val rang : int * int -> int * int -> int = <fun>


- : int = 13


In [5]:
(* 5. *)
let sommets (p, q) i =
    if i < p*(q - 1) then
        let u = i/(q - 1) + p*(i mod (q - 1)) in
        u, u + p
    else 
        let j = i - p*(q - 1) in
        let u = j + j/p in
        u, u + 1;;
    
sommets (4, 3) 4;;
sommets (4, 3) 16;;

val sommets : int * int -> int -> int * int = <fun>


- : int * int = (2, 6)


- : int * int = (10, 11)


In [6]:
(* 6. *)
let quadrillage p q =
    let n = p*q in
    let g = Array.make n [] in
    for i = 0 to n - 1 do
        if i mod p <> 0 then g.(i) <- (i - 1)::g.(i);
        if (i + 1) mod p <> 0 then g.(i) <- (i + 1)::g.(i);
        if i/p <> 0 then g.(i) <- (i - p)::g.(i);
        if i/p <> q - 1 then g.(i) <- (i + p)::g.(i)
    done;
    g;;
    
quadrillage 4 3

val quadrillage : int -> int -> int list array = <fun>


- : int list array =
[|[4; 1]; [5; 2; 0]; [6; 3; 1]; [7; 2]; [8; 0; 5]; [9; 1; 6; 4];
  [10; 2; 7; 5]; [11; 3; 6]; [4; 9]; [5; 10; 8]; [6; 11; 9]; [7; 10]|]


## Caractérisation des arbres

In [7]:
(* 11. *)
let rec representant p s =
    if p.(s) < 0 then s
    else representant p p.(s);;

let p = [| -2; 5; 0; 7; 7; 6; -3; -2; 6 |];;
representant p 1

val representant : int array -> int -> int = <fun>


val p : int array = [|-2; 5; 0; 7; 7; 6; -3; -2; 6|]


- : int = 6


In [8]:
(* 12. *)
let union p s t =
    if p.(s) = p.(t) then p.(t) <- p.(t) - 1;
    if p.(s) < p.(t) then p.(t) <- s
    else p.(s) <- t;;
    
union p 0 7; p

val union : int array -> int -> int -> unit = <fun>


- : int array = [|7; 5; 0; 7; 7; 6; -3; -3; 6|]


In [9]:
(* 15. *)
exception Arbre;;

let est_un_arbre g =
    let n = Array.length g in
    let p = Array.init n (fun i -> -1) in
    try for i = 0 to n - 1 do
            let ri = representant p i in
            List.iter (fun j -> 
                if j > i then
                let rj = representant p j in
                if ri = rj then raise Arbre
                else union p ri rj) g.(i)
        done;
        nombre_aretes g = n - 1
    with Arbre -> false;;
    
not (est_un_arbre [| []; [3; 4]; [3]; [1; 2; 4; 4]; [1; 3; 3] |])
&& est_un_arbre [| [1]; [0; 3]; [3]; [1; 2; 4]; [3] |]

exception Arbre


val est_un_arbre : int list array -> bool = <fun>


- : bool = true


In [10]:
(* 18. *)
type chemin = {debut : int; mutable fin : int; suivant : int array};;

let marche_aleatoire adj parent s =
    let c = {
        debut = s;
        fin = s;
        suivant = Array.make (Array.length adj) (-1)
    } in
    while parent.(c.fin) = -2 do
        let i = Random.int (Array.length adj.(c.fin)) in
        let v = adj.(c.fin).(i) in
        c.suivant.(c.fin) <- v;
        c.fin <- v;
    done;
    c

type chemin = { debut : int; mutable fin : int; suivant : int array; }


val marche_aleatoire : int array array -> int array -> int -> chemin = <fun>


In [11]:
(* 19. *)
let rec greffe parent c =
    let u = ref c.debut in
    while !u <> c.fin do
        let v = c.suivant.(!u) in
        parent.(!u) <- v;
        u := v
    done

val greffe : int array -> chemin -> unit = <fun>


In [12]:
(* 20. *)
let wilson g r =
    let adj = adjacence g in
    let parent = Array.make (Array.length g) (-2) in
    parent.(r) <- -1;
    for s = 0 to Array.length parent - 1 do
        if parent.(s) = -2 then
            let c = marche_aleatoire adj parent s in
            greffe parent c
    done;
    parent;;
    
wilson g32_list 0

val wilson : int list array -> int -> int array = <fun>


- : int array = [|-1; 0; 1; 4; 1; 4|]


## Arbres couvrants et pavages par des dominos

In [13]:
type direction = S | W | N | E;;
let p = 4;;
let q = 3;;

type direction = S | W | N | E


val p : int = 4


val q : int = 3


In [14]:
(* 24. *)
let coord_noire i = 2*(i mod p), 2*(i/p);;

coord_noire 6

val coord_noire : int -> int * int = <fun>


- : int * int = (4, 2)


In [15]:
(* 25. *)
let sommet_direction s d = 
    let i, j = coord_noire s in
    match d with
        | N -> if j = q - 1 then -1 else s + p
        | S -> if j = 0 then -1 else s - p
        | W -> if i = 0 then -1 else s - 1
        | E -> if i = p - 1 then -1 else s + 1;;
        
sommet_direction 5 W

val sommet_direction : int -> direction -> int = <fun>


- : int = 4


In [16]:
(* 26. *)
let phi pavage =
    let n = Array.length pavage in
    let parent = Array.make n (-2) in
    for i = 0 to n - 1 do
        parent.(i) <- sommet_direction i pavage.(i)
    done;
    parent

val phi : direction array -> int array = <fun>


In [17]:
(* 27. *)

let dual () =
    let n' = (p - 1)*(q - 1) + 1 in
    let g' = Array.make n' [] in
    let g = quadrillage (p - 1) (q - 1) in
    for i = 0 to n' - 2 do
        g'.(i + 1) <- List.map ((+) 1) g.(i)
    done;
    let add i = 
        g'.(i) <- 0::g'.(i);
        g'.(0) <- i::g'.(0) in
    for i = 1 to p - 1 do add i done; (* bord du bas *)
    for i = n' - p + 1 to n' - 1 do add i done; (* bord du haut *)
    for i = 1 to q - 1 do add (i*(p - 1)) done; (* bord droit *)
    for i = 1 to q - 1 do add (1 + (i - 1)*(p - 1)) done; (* bord gauche *)
    g'

val dual : unit -> int list array = <fun>


In [18]:
let g_etoile = dual ()

val g_etoile : int list array =
  [|[4; 1; 6; 3; 6; 5; 4; 3; 2; 1]; [0; 0; 4; 2]; [0; 5; 3; 1]; [0; 0; 6; 2];
    [0; 0; 1; 5]; [0; 2; 6; 4]; [0; 0; 3; 5]|]
